In [1]:
import sys
freecadpath = "/home/julian/extrepositories/FreeCAD/freecad-build/lib"
sys.path.append(freecadpath)
import FreeCAD
import PathScripts.PathJob as PathJob
import PathScripts.PathLog as PathLog
import PathScripts.PathToolController as PathToolController
import PathScripts.PathUtil as PathUtil


In [2]:
def extractpostlistfromjob(job):
    postlist = []
    currTool = None
    for obj in job.Operations.Group:
        PathLog.debug("obj: {}".format(obj.Name))
        tc = PathUtil.toolControllerForOp(obj)
        if tc is not None:
            if tc.ToolNumber != currTool:
                postlist.append(tc)
                currTool = tc.ToolNumber
        postlist.append(obj)
    return postlist

def extractjobslistfromfile(fname):
    doc = FreeCAD.open(fname)
    jobs = [o  for o in FreeCAD.ActiveDocument.Objects  if hasattr(o, "Proxy") and isinstance(o.Proxy, PathJob.ObjectJob)]
    return jobs

def extractpostlistfromfile(job):
    doc = FreeCAD.open(fname)
    job = doc.getObject("Job")
    return extractpostlistfromjob(job)


In [3]:
fname = "/home/julian/data/freecad/swanneck3.fcstd"
jobs = extractjobslistfromfile(fname)
#postlist = extractpostlistfromfile(fname)
#postlist = extractpostlistfromfile(fname)
print(jobs)

[<Path::FeaturePython object>, <Path::FeaturePython object>]


In [4]:
postlist = extractpostlistfromjob(jobs[0])
print(postlist)

[<Path::FeaturePython object>, <Path::FeaturePython object>, <Path::FeaturePython object>]


In [5]:
from kineticnc_post import flattenandgroup
tooldefmotions = flattenandgroup(postlist)


kineticnc_post gcode postprocessor loaded.


In [6]:
tooldefmotions[0][0]

{'Diameter': '4.0',
 'S': 18000.0,
 'T': 1.0,
 'firstpos': {'F': 200.0, 'X': 105.959169, 'Y': 162.669622, 'Z': 30.0},
 'lastpos': {'F': 950.0, 'X': 105.959169, 'Y': 162.669622, 'Z': 30.0}}

In [7]:
from kineticnc_post import flattenobjectlisttocommands
cmds = flattenobjectlisttocommands(postlist)
def extractfirstpos(cmds):
    pos = { }
    for cmd in cmds:
        for w in "XYZF":
            if w not in pos and w in cmd.Parameters:
                pos[w] = cmd.Parameters[w]
        if len(pos) == 4:
            break
    return pos
extractfirstpos(cmds)

{'F': 200.0, 'X': 105.959169, 'Y': 162.669622, 'Z': 30.0}

In [7]:
from StringIO import StringIO
from kineticnc_post import writetooldefheader, writemotioncmds, writefilefooter

fout = StringIO()
currpos = {}
for tooldef, motioncmds in tooldefmotions:
    tooldef["filename"] = "hi"
    writetooldefheader(fout, tooldef, currpos)
    writemotioncmds(fout, motioncmds, currpos)
writefilefooter(fout, currpos)
